In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import pickle

In [2]:
# Load your dataset
data = pd.read_csv('student_data_with_gpa.csv')

In [3]:
# Preprocess the Data
X = data.drop('GPA', axis=1)
y = data['GPA']

In [4]:
# Define preprocessing pipeline
numeric_features = ['Age', 'CognitiveScore', 'LearningStrategyScore']
categorical_features = ['Gender', 'Discipline']

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [5]:
# Preprocess data
X_preprocessed = preprocessor.fit_transform(X)

In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

In [7]:
# Build the Neural Network Model
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))  # Use Input layer to specify input shape
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))  # Linear activation for regression

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error'])

In [8]:
# Train the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - loss: 14.4519 - mean_absolute_error: 3.6678 - val_loss: 10.8885 - val_mean_absolute_error: 3.1727
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 10.6949 - mean_absolute_error: 3.1020 - val_loss: 7.4629 - val_mean_absolute_error: 2.5953
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7.2878 - mean_absolute_error: 2.4968 - val_loss: 4.2030 - val_mean_absolute_error: 1.9156
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 4.4610 - mean_absolute_error: 1.8355 - val_loss: 1.7573 - val_mean_absolute_error: 1.1951
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 3.1829 - mean_absolute_error: 1.4878 - val_loss: 0.7652 - val_mean_absolute_error: 0.7313
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.4168 - mean_absolute_error: 1.2683 - val_loss: 0.6457 - val_mean_absolute_error: 0.6657
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2.3004 - mean_absolute_error: 1.2089 - val_loss:

In [9]:
# Evaluate the Model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test MAE: {mae:.2f}')

Test MAE: 0.51


In [11]:
# Save the model and preprocessor for use in the Streamlit app
model.save('student_gpa_model.h5')
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)